In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, auc
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler 

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, confusion_matrix
from sklearn.tree import plot_tree

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import torch
from torch import nn
from sklearn.datasets import make_blobs
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
hotels = pd.read_csv("hotel_bookings.csv")

In [3]:
hotels.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
hotels_cop = hotels.copy()

In [5]:
print("# of NaN in each columns:", hotels.isnull().sum(), sep='\n')

# of NaN in each columns:
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                          

In [6]:
hotels_cop = hotels_cop.drop(['company'], axis = 1)

In [7]:
hotels_cop['children'] = hotels_cop['children'].fillna(0)

In [8]:
hotels_cop['hotel'] = hotels_cop['hotel'].map({'Resort Hotel':0, 'City Hotel':1})

hotels_cop['arrival_date_month'] = hotels_cop['arrival_date_month'].map({'January':1, 'February': 2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7,
                                                            'August':8, 'September':9, 'October':10, 'November':11, 'December':12})

In [9]:
def is_family(data):
    if ((data['adults'] > 0) & (data['children'] > 0)):
        val = 1
    elif ((data['adults'] > 0) & (data['babies'] > 0)):
        val = 1
    else:
        val = 0
    return val

def did_deposit(data):
    if ((data['deposit_type'] == 'No Deposit') | (data['deposit_type'] == 'Refundable')):
        return 0
    else:
        return 1

In [10]:
hotels_cop["is_family"] = hotels_cop.apply(is_family, axis = 1)
hotels_cop["total_customer"] = hotels_cop["adults"] + hotels_cop["children"] + hotels_cop["babies"]
hotels_cop["deposit_given"] = hotels_cop.apply(did_deposit, axis=1)
hotels_cop["total_nights"] = hotels_cop["stays_in_weekend_nights"]+ hotels_cop["stays_in_week_nights"]

In [11]:
hotels_cop = hotels_cop.drop(columns = ['adults', 'babies', 'children', 'deposit_type', 'reservation_status_date'])

In [12]:
hotels_cop = pd.get_dummies(data = hotels_cop, columns = ['meal', 'market_segment', 'distribution_channel',
                                            'reserved_room_type', 'assigned_room_type', 'customer_type', 'reservation_status'])

In [13]:
indices = hotels_cop.loc[pd.isna(hotels_cop["country"]), :].index 
hotels_cop = hotels_cop.drop(hotels_cop.index[indices])   
hotels_cop = hotels_cop.drop(columns = ['arrival_date_week_number', 'stays_in_weekend_nights', 'arrival_date_month', 'agent'], axis = 1)

In [14]:
le = LabelEncoder()

In [15]:
hotels_cop['country'] = le.fit_transform(hotels_cop['country']) 

In [16]:
hotels_cop = hotels_cop.drop(columns = ['reservation_status_Canceled', 'reservation_status_Check-Out', 'reservation_status_No-Show'], axis = 1)

In [17]:
hotels_cop.to_csv('hotels_cop.csv', index=False)